# ABS Quarterly Job Vacancies 6354

## Python set-up

In [1]:
# system imports
import textwrap
from typing import Any, cast

# analytic imports
import pandas as pd
from readabs import metacol
import readabs as ra

# local imports
from abs_helper import get_abs_data
from abs_plotting import plot_rows_individually
from abs_summary_plot import plot_summary
from plotting import line_plot, plot_covid_recovery

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("6354.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

In [3]:
# list tables
textwrap.wrap(", ".join(abs_dict.keys()), width=80)

['6354001, 6354002, 6354003, 6354004']

## Plot

### Headline charts

In [4]:
def plot_headline() -> None:
    """Produce headline charts."""

    # get mean quarterly Labour force numbers for Australia
    lfs, _ = ra.read_abs_series("6202.0", "A84423047L", single_excel_only="6202001")
    lf = ra.monthly_to_qtly(lfs["A84423047L"], q_ending="NOV", f="mean")

    # get headline job vacancy rate and job vacancies
    stype = "Seasonally Adjusted"
    selector: dict[str, str] = {
        stype: metacol.stype,
        "6354001": metacol.table,
        "Job Vacancies ;  Australia ;": metacol.did,
    }
    table, ident, units = ra.find_abs_id(meta, selector)
    jva = abs_dict[table][ident]
    # surface missing data in the job vacancy series
    jva = jva.reindex(pd.period_range(start=jva.index[0], end=lf.index[-1]))
    jvr = jva / lf * 100

    # plotting
    common: dict[str, Any] = {
        "rfooter": source,
        "lfooter": f"Australia. {stype.capitalize()} series. ",
        "show": SHOW,
    }
    for data, ylabel, title in zip(
        (jvr, jva),
        ("Job Vacancy Rate (%)", "Thousands"),
        ("Job Vacancy Rate", "Job Vacancies"),
    ):
        also = common.copy()
        if "Rate" in title:
            also["lfooter"] += "Job Vacancy Rate = Job Vacancies / Labour Force. "
        also["ylabel"] = ylabel
        also["title"] = title

        line_plot(
            data,
            dropna=False,
            **also,
        )
        plot_covid_recovery(
            data,
            start_r="2009Q4",
            end_r="2019Q4",
            tags="-covid",
            **also,
        )


plot_headline()

### Industry sector charts

In [5]:
def plot_industry() -> None:
    """Plot industry data."""

    stype = "Original"
    selector = {
        stype: metacol.stype,
        "^Job Vacancies": metacol.did,
        "6354004": metacol.table,
    }

    common: dict[str, Any] = {
        "selector": selector,
        "rfooter": source,
        "regex": True,
        "lfooter": f"Australia. {stype.capitalize()} series. ",
        "show": SHOW,
    }

    plot_rows_individually(
        abs_dict,
        meta,
        plot_function=plot_covid_recovery,
        tags="Covid Recovery",
        **common,
    )

    plot_rows_individually(
        abs_dict,
        meta,
        plot_function=plot_covid_recovery,
        tags="Covid Recovery long run",
        start_r="2000Q1",
        end_r="2019Q4",
        **common,
    )


plot_industry()

### State charts

In [6]:
def plot_state() -> None:
    """Plot state data."""

    stype = "Original"
    selector = {
        stype: metacol.stype,
        "^Job Vacancies": metacol.did,
        "Original": metacol.stype,
        "6354001": metacol.table,
    }

    common: dict[str, Any] = {
        "selector": selector,
        "regex": True,
        "rfooter": source,
        "lfooter": f"Australia. {stype.capitalize()} series. ",
        "show": SHOW,
    }

    plot_rows_individually(
        abs_dict,
        meta,
        plot_function=plot_covid_recovery,
        tags="Covid Recovery",
        **common,
    )


plot_state()

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Jul 16 2024 09:52:15

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

pandas : 2.2.2
readabs: 0.0.10a4

Watermark: 2.4.3



In [8]:
print("Finished")

Finished
